In [18]:
import numpy as np
from astropy.coordinates import EarthLocation, SkyCoord, AltAz
from astropy.time import Time
import astropy.units as u
from astroplan import Observer
from datetime import datetime
from tqdm import tqdm

def check_observability_midnight(astropy_table, start_date, end_date, location=None):
    """
    Check which sources in an Astropy table are observable with airmass < 1.5
    at the middle of the night, where "middle of the night" is defined as the
    median time between sunset and sunrise. Also, for each source, returns the month
    when it is best observed (i.e. when it reaches its minimum airmass at that time).

    Parameters
    ----------
    astropy_table : astropy.table.Table
        An Astropy Table containing at least the columns 'ra', 'dec', and 'name'.
    start_date : str
        Start date in 'dd-mm-yyyy' format, e.g. '01-09-2025'.
    end_date : str
        End date in 'dd-mm-yyyy' format, e.g. '31-03-2026'.
    location : astropy.coordinates.EarthLocation, optional
        The observer's location. If None, defaults to Mauna Kea, Hawaii.
        (Default timezone is set accordingly.)

    Returns
    -------
    astropy.table.Table
        The input table with two additional columns:
         - 'observable': True if the source reaches an airmass < 1.5 at some middle-of-night time.
         - 'best_month': The month (as a string, e.g. 'September') when the source is best observed,
                         or None if the source never rises above the horizon at the computed times.
    """
    # Use default location (Mauna Kea) if none is provided.
    if location is None:
        location = EarthLocation(lat=19.8207*u.deg, lon=-155.4681*u.deg, height=4205*u.m)
        timezone = 'US/Hawaii'
    else:
        timezone = 'UTC'
    
    observer = Observer(location=location, timezone=timezone)
    
    # Parse the start and end dates from the provided "dd-mm-yyyy" format.
    start_dt = datetime.strptime(start_date, '%d-%m-%Y')
    end_dt = datetime.strptime(end_date, '%d-%m-%Y')
    
    # Convert the datetime objects into astropy Time objects.
    t_start = Time(start_dt)
    t_end = Time(end_dt)
    
    # Generate a list of days between t_start and t_end.
    n_days = int((t_end - t_start).to(u.day).value) + 1
    days = t_start + np.arange(n_days)*u.day

    # For each day, compute the middle of the night (the midpoint between sunset and the following sunrise).
    middle_night_times = []
    for day in days:
        try:
            # Calculate sunset time for the day.
            sunset = observer.sun_set_time(day, which='next')
            # Calculate the following sunrise after sunset.
            sunrise = observer.sun_rise_time(sunset, which='next')
            # Define middle of the night as the midpoint between sunset and sunrise.
            middle_night = sunset + (sunrise - sunset)/2
            middle_night_times.append(middle_night)
        except Exception as e:
            # Skip days where sunset/sunrise cannot be computed.
            continue
    
    observable_list = []
    best_month_list = []
    
    # Evaluate each source at all computed middle-of-night times.
    for row in tqdm(astropy_table):
        coord = SkyCoord(ra=row['ra']*u.deg, dec=row['dec']*u.deg)
        best_airmass = np.inf
        best_time = None
        
        for mid in middle_night_times:
            altaz = coord.transform_to(AltAz(obstime=mid, location=location))
            # Only consider times when the source is above the horizon.
            if altaz.alt < 0*u.deg:
                continue
            airmass = altaz.secz
            if np.isfinite(airmass) and airmass < best_airmass:
                best_airmass = airmass
                best_time = mid
        
        # A source is considered observable if its best airmass is below 1.5.
        observable_list.append(best_airmass < 1.5)
        # Record the month corresponding to the best (lowest) airmass.
        if best_time is not None:
            best_month = best_time.datetime.strftime('%B')
        else:
            best_month = None
        best_month_list.append(best_month)
    
    # Append the results to the input table.
    astropy_table['observable'] = observable_list
    astropy_table['best_month'] = best_month_list
    
    return astropy_table


In [16]:
import numpy as np
from astropy.coordinates import EarthLocation, SkyCoord, AltAz
from astropy.time import Time
import astropy.units as u
from astroplan import Observer
from datetime import datetime
import concurrent.futures
from functools import partial
from tqdm import tqdm  # progress bar

def _compute_best_for_star(ra, dec, location, middle_night_times):
    """
    Compute the best (lowest) airmass and corresponding middle-of-night time for a given star.
    
    Parameters
    ----------
    ra : float
        Right ascension in degrees.
    dec : float
        Declination in degrees.
    location : EarthLocation
        Observer's location.
    middle_night_times : list of astropy Time
        Precomputed list of middle-of-night times.
    
    Returns
    -------
    tuple(bool, str)
        A tuple containing:
          - observable: True if the star's best airmass is below 1.5.
          - best_month: Name of the month (e.g., 'September') corresponding to the time
                        of best airmass, or None if the star never rises.
    """
    coord = SkyCoord(ra=ra*u.deg, dec=dec*u.deg)
    best_airmass = np.inf
    best_time = None
    for mid in middle_night_times:
        altaz = coord.transform_to(AltAz(obstime=mid, location=location))
        # Consider only times when the source is above the horizon.
        if altaz.alt < 0*u.deg:
            continue
        airmass = altaz.secz
        if np.isfinite(airmass) and airmass < best_airmass:
            best_airmass = airmass
            best_time = mid
    observable = best_airmass < 1.5
    best_month = best_time.datetime.strftime('%B') if best_time is not None else None
    return (observable, best_month)

def check_observability_midnight(astropy_table, start_date, end_date, location=None):
    """
    Check which sources in an Astropy table are observable with airmass < 1.5
    at the middle of the night, where "middle of the night" is defined as the
    median time between sunset and sunrise. For each source, also return the month
    when it is best observed (i.e. when it reaches its minimum airmass at that time).

    This version uses multiprocessing to process each star in parallel and
    displays a progress bar.

    Parameters
    ----------
    astropy_table : astropy.table.Table
        An Astropy Table containing at least the columns 'ra', 'dec', and 'name'.
    start_date : str
        Start date in 'dd-mm-yyyy' format (e.g. '01-09-2025').
    end_date : str
        End date in 'dd-mm-yyyy' format (e.g. '31-03-2026').
    location : astropy.coordinates.EarthLocation, optional
        Observer's location. If None, defaults to Mauna Kea, Hawaii.

    Returns
    -------
    astropy.table.Table
        The input table with two additional columns:
          - 'observable': True if the source reaches an airmass < 1.5 at some middle-of-night time.
          - 'best_month': Month (as a string, e.g. 'September') when the source is best observed,
                          or None if the source never rises above the horizon at the computed times.
    """
    # Default location: Mauna Kea, Hawaii.
    if location is None:
        location = EarthLocation(lat=19.8207*u.deg, lon=-155.4681*u.deg, height=4205*u.m)
        timezone = 'US/Hawaii'
    else:
        timezone = 'UTC'
    
    observer = Observer(location=location, timezone=timezone)
    
    # Parse the start and end dates from 'dd-mm-yyyy' format.
    start_dt = datetime.strptime(start_date, '%d-%m-%Y')
    end_dt = datetime.strptime(end_date, '%d-%m-%Y')
    t_start = Time(start_dt)
    t_end = Time(end_dt)
    
    # Create a list of days in the date range.
    n_days = int((t_end - t_start).to(u.day).value) + 1
    days = t_start + np.arange(n_days)*u.day

    # Compute the middle-of-night times: the midpoint between sunset and the following sunrise.
    middle_night_times = []
    for day in days:
        try:
            sunset = observer.sun_set_time(day, which='next')
            sunrise = observer.sun_rise_time(sunset, which='next')
            middle_night = sunset + (sunrise - sunset) / 2
            middle_night_times.append(middle_night)
        except Exception:
            # Skip the day if sunset/sunrise times cannot be computed.
            continue

    # Prepare lists of RA and Dec for each source.
    ra_list = [row['ra'] for row in astropy_table]
    dec_list = [row['dec'] for row in astropy_table]
    
    # Use multiprocessing to compute observability for each source.
    func = partial(_compute_best_for_star, location=location, middle_night_times=middle_night_times)
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # Wrap the executor.map with tqdm for a progress bar.
        results = list(tqdm(executor.map(func, ra_list, dec_list), total=len(ra_list), desc="Processing stars"))
    
    # Unpack results and add as new columns to the table.
    observable_list, best_month_list = zip(*results) if results else ([], [])
    astropy_table['observable'] = observable_list
    astropy_table['best_month'] = best_month_list
    
    return astropy_table


In [11]:
location_paranal = EarthLocation(lat=-24.627222222222*u.deg, lon=-70.404166666667*u.deg, height=2635*u.m)

In [20]:
from astropy.table import Table
candidates = Table.read('/Users/mncavieres/Documents/2024-2/HVS/Data/candidates/high_likelihood_candidates.dat', format='ascii')

In [21]:
candidates.rename_column('col1', 'name')
candidates.rename_column('col2', 'ra')
candidates.rename_column('col3', 'dec')

In [22]:
candidates = check_observability_midnight(candidates, '01-10-2025', '30-04-2026', location=location_paranal)

100%|██████████| 488/488 [04:03<00:00,  2.00it/s]
